In [ ]:
!pip install PyMuPDF

In [ ]:
# set filename here 
csvfilename = "Forwardlookinginfo.csv"


In [ ]:
import os
import re
import fitz 
import pandas as pd

In [ ]:
#!pip install striprtf

# Read keyword files and build lists of keywords

In [ ]:
keyworddf = pd.read_csv('keywordfileneverdeleteit.csv')

# Example for two types of keywords with case-sentivity 

type1caseinsensitive = keyworddf[(keyworddf['Step']==1)&(keyworddf['Type']=='1')&(
    keyworddf['Casesensitivity']==0)]['Keywords'].to_list()
type1casesensitive = keyworddf[(keyworddf['Step']==1)&(keyworddf['Type']=='1')&(
    keyworddf['Casesensitivity']==1)]['Keywords'].to_list()


type2caseinsensitive = keyworddf[(keyworddf['Step']==1)&(keyworddf['Type']=='2')&(
    keyworddf['Casesensitivity']==0)]['Keywords'].to_list()
type2casesensitive = keyworddf[(keyworddf['Step']==1)&(keyworddf['Type']=='2')&(
    keyworddf['Casesensitivity']==1)]['Keywords'].to_list()

#step2
typeOperationSensitive = keyworddf[(keyworddf['Step']==2)&(keyworddf['Type']=='Operation')&(
    keyworddf['Casesensitivity']==0)]['Keywords'].to_list()
typeOperationInsensitive = keyworddf[(keyworddf['Step']==2)&(keyworddf['Type']=='Operation')&(
    keyworddf['Casesensitivity']==1)]['Keywords'].to_list()

typeFinanceSensitive = keyworddf[(keyworddf['Step']==2)&(keyworddf['Type']=='Finance')&(
    keyworddf['Casesensitivity']==0)]['Keywords'].to_list()
typeFinanceInsensitive = keyworddf[(keyworddf['Step']==2)&(keyworddf['Type']=='Finance')&(
    keyworddf['Casesensitivity']==1)]['Keywords'].to_list()

typeAccountingSensitive = keyworddf[(keyworddf['Step']==2)&(keyworddf['Type']=='Accounting')&(
    keyworddf['Casesensitivity']==0)]['Keywords'].to_list()
typeAccountingInsensitive = keyworddf[(keyworddf['Step']==2)&(keyworddf['Type']=='Accounting')&(
    keyworddf['Casesensitivity']==1)]['Keywords'].to_list()


In [ ]:
def step1(sentences,year):
    
    matchingsentences = []

    for sentence in sentences:

        text = sentence

        # type 1
        for keyword in type1caseinsensitive:
            if re.search('(^| )'+keyword+r'($|\.| )',text,
                       re.I) !=None:
                matchingsentences.append({'Step':1,'Type':1,'Keyword':keyword,'Sentence':text})
        
        for keyword in type1casesensitive:
            if re.search('(^| )'+keyword+r'($|\.| )',text
                       ) !=None:
                matchingsentences.append({'Step':1,'Type':1,'Keyword':keyword,'Sentence':text})
        # type 2
        for keyword in type2caseinsensitive:
            if re.search('(^| )'+keyword+r'($|\.| )',text,
                       re.I) !=None:
                matchingsentences.append({'Step':1,'Type':2,'Keyword':keyword,'Sentence':text})

        for keyword in type2casesensitive:
            if re.search('(^| )'+keyword+r'($|\.| )',text
                       ) !=None:
                matchingsentences.append({'Step':1,'Type':2,'Keyword':keyword,'Sentence':text})
        
        # type 3       
        if re.search('(^| )'+year+r'($|\.| )',text,
                   re.I) !=None:
            matchingsentences.append({'Step':1,'Type':3,'Keyword':keyword,'Sentence':text})
            
    
    return pd.DataFrame(matchingsentences)

In [ ]:
def step2(sentences):
    

    matchingsentences = []
    
    caught = 0

    for text in sentences:

        for keyword in typeOperationSensitive:
            if re.search('(^| )'+keyword+r'($|\.| )',text,
                       re.I) !=None:
                matchingsentences.append({'Step':2,'Type':'Operation','Keyword':keyword,'Sentence':text})
                caught = 1

        for keyword in typeOperationInsensitive:
            if re.search('(^| )'+keyword+r'($|\.| )',text
                       ) !=None:
                matchingsentences.append({'Step':2,'Type':'Operation','Keyword':keyword,'Sentence':text})
                caught = 1

        for keyword in typeFinanceSensitive:
            if re.search('(^| )'+keyword+r'($|\.| )',text,
                       re.I) !=None:
                matchingsentences.append({'Step':2,'Type':'Finance','Keyword':keyword,'Sentence':text})
                caught = 1

        for keyword in typeFinanceInsensitive:
            if re.search('(^| )'+keyword+r'($|\.| )',text
                       ) !=None:
                matchingsentences.append({'Step':2,'Type':'Finance','Keyword':keyword,'Sentence':text})
                caught = 1

        for keyword in typeAccountingSensitive:
            if re.search('(^| )'+keyword+r'($|\.| )',text,
                       re.I) !=None:
                matchingsentences.append({'Step':2,'Type':'Accounting','Keyword':keyword,'Sentence':text})
                caught = 1

        for keyword in typeAccountingInsensitive:
            if re.search('(^| )'+keyword+r'($|\.| )',text
                       ) !=None:
                matchingsentences.append({'Step':2,'Type':'Accounting','Keyword':keyword,'Sentence':text})
                caught = 1
                
        if caught == 0:
            matchingsentences.append({'Step':2,'Type':'NoMatch','Keyword':keyword,'Sentence':text})
                
        
    return pd.DataFrame(matchingsentences)

# Read directory and find all RTFs

In [ ]:
# This cell takes time. Wait untile the * change to a number

dfs = []
for foldername in os.listdir():
    foldercounter = 0
    if '.' in foldername:
        continue
    else:
        for pdffile in os.listdir('./'+foldername):
            
            if pdffile[-3:]=='pdf':
                
                doc = fitz.open('./'+foldername+'/'+pdffile)
                sentences = []
                for i in range(doc.pageCount):
                    sentences += re.split(r"\. ", doc.getPageText(i))

                result = pd.concat([
                    step1(sentences,str(int(re.findall("(20[0-9][0-9])",pdffile)[0])+1)),
                    step2(sentences)])
                result['Company']= foldername
                result['Report'] = pdffile
                dfs.append(result)
#                print (foldername+'/'+rtffile+' has been processed')
                foldercounter+=1
        print (foldername+' has been processed. In total: ' + str(foldercounter)+' files')


In [ ]:
pd.concat(dfs).to_csv(csvfilename, index = False)